# Instalación de Librerias

In [ ]:
!pip install playwright openpyxl
!playwright install

In [ ]:
!pip install -U playwright
!playwright install

# Ingresar los códigos padres

```
# Celda 1: Bloque adicional para ingresar todos los códigos padres
codigos_input = """
10923690001

"""  # Reemplazar con los códigos pegados desde un Excel, separados por espacios
codigos = codigos_input.strip().split()

```

In [ ]:
# Celda 1: Bloque adicional para ingresar todos los códigos padres
codigos_input = """
10803421016
10962435018
10962438016
10962595019
10875176015
10911276014
10966099022
10962821013
10911276017
10962822013
10962821014
10866961002
10962821019
10962438013
10962692014
10875176013
10962435019
10962822016
10962438018
10966099018
10962594013
10962736022
10962389018
10930745010
10962389016
"""  # Reemplazar con los códigos pegados desde un Excel, separados por espacios
codigos = codigos_input.strip().split()

# Script principal

In [ ]:
# Celda 3: Script principal mejorado para la extracción de precios

import nest_asyncio
import asyncio
from datetime import datetime
from playwright.async_api import async_playwright
from openpyxl import Workbook
from threading import Lock
from google.colab import files  # Importar para descargar el archivo en Colab

# Permitir anidar el loop de eventos
nest_asyncio.apply()

# Variables globales
proceso_en_ejecucion = False
estado_codigos = []
total_codigos = 0
codigos_procesados = 0
lock = Lock()  # Para controlar el acceso concurrente a las variables globales
max_retries = 3  # Número máximo de reintentos
max_concurrent_requests = 10  # Máximo número de solicitudes concurrentes

# Semáforo para limitar el número de solicitudes concurrentes
semaphore = asyncio.Semaphore(max_concurrent_requests)

# Función asíncrona para obtener el estado de un producto, su precio y la cantidad de imágenes usando Playwright
async def obtener_estado_y_precio(codigo_padre, reintento=0):
    async with semaphore:
        url_base = f'https://www.marathon.cl/{codigo_padre}.html'
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            try:
                await page.goto(url_base, timeout=60000)  # Tiempo de espera de 60 segundos
                await page.wait_for_selector("body", timeout=10000)  # Esperar a que el cuerpo de la página se cargue

                # Verificar si la página redirige al inicio (indicador: URL de redirección)
                if page.url == "https://www.marathon.cl/home/":
                    await browser.close()
                    return codigo_padre, "Web no encontrada", "Precio no disponible", "Cantidad de imágenes no disponible"

                # Buscar los botones de talla y determinar si están seleccionados
                botones_talla = await page.query_selector_all("button.size-attribute.swatchable.selectable.swatch-square")
                estado = "Agotado"
                for boton in botones_talla:
                    if "selected-assistive-text" in await boton.inner_html():
                        estado = "Disponible" if not await boton.is_disabled() else "Agotado"
                        break

                # Extraer el precio utilizando el selector proporcionado
                try:
                    precio_element = await page.query_selector('div.price > span.space-text > span.sales > span.value')
                    precio = await precio_element.inner_text() if precio_element else "Precio no disponible"
                except:
                    precio = "Precio no disponible"
                
                # Contar la cantidad de imágenes
                try:
                    imagenes = await page.query_selector_all('img.galley_img')
                    cantidad_imagenes = len(imagenes)
                except:
                    cantidad_imagenes = "Cantidad de imágenes no disponible"
                
                await browser.close()
                return codigo_padre, estado, precio, cantidad_imagenes
            except Exception as e:
                await browser.close()
                if reintento < max_retries:
                    print(f"Reintento {reintento+1} para {codigo_padre} debido a {e}")
                    return await obtener_estado_y_precio(codigo_padre, reintento + 1)  # Reintentar
                else:
                    return codigo_padre, f"Error: {e}", "Precio no disponible", "Cantidad de imágenes no disponible"

# Función asíncrona para procesar los códigos y guardar en Excel
async def procesar_codigos(codigos):
    global proceso_en_ejecucion, estado_codigos, total_codigos, codigos_procesados
    total_codigos = len(codigos)
    start_time = datetime.now()

    async def obtener_estado_concurrente(codigo_padre):
        return await obtener_estado_y_precio(codigo_padre)
    
    tasks = [obtener_estado_concurrente(codigo) for codigo in codigos]
    for task in asyncio.as_completed(tasks):
        if not proceso_en_ejecucion:
            break
        codigo_padre, estado, precio, cantidad_imagenes = await task
        with lock:
            estado_codigos.append((codigo_padre, estado, precio, cantidad_imagenes))
            codigos_procesados += 1

        # Actualizar información en la consola
        elapsed_time = datetime.now() - start_time
        tiempo_transcurrido = str(elapsed_time).split('.')[0]  # Formato HH:MM:SS
        print(f"Procesando código {codigos_procesados}/{total_codigos} - Tiempo transcurrido: {tiempo_transcurrido}")

    if proceso_en_ejecucion:
        guardar_resultados()

# Función para pausar el proceso
def pausar_proceso():
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    print("Proceso pausado. Puede continuar luego.")

# Función para detener el proceso
def detener_proceso():
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    guardar_resultados()
    print("Proceso detenido.")

# Función para guardar los resultados en Excel
def guardar_resultados():
    global proceso_en_ejecucion
    # Guardar en Excel
    wb = Workbook()
    ws = wb.active
    ws.title = "Control Stock Web"
    ws['A1'] = "CODIGO"
    ws['B1'] = "STATUS WEB"
    ws['C1'] = "PRECIO"
    ws['D1'] = "Cant. Img"
    for i, (codigo, estado, precio, cantidad_imagenes) in enumerate(estado_codigos, start=2):
        ws[f'A{i}'] = codigo
        ws[f'B{i}'] = estado
        ws[f'C{i}'] = precio
        ws[f'D{i}'] = cantidad_imagenes

    # Guardar archivo
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo = f"Control Stock Web {fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    print(f"Archivo '{nombre_archivo}' guardado con los estados, precios y cantidad de imágenes de los productos.")
    # Descarga el archivo de Excel en Colab
    files.download(nombre_archivo)

# Función para iniciar el procesamiento
async def iniciar_procesamiento(codigos):
    global proceso_en_ejecucion, codigos_procesados, estado_codigos
    if not proceso_en_ejecucion:
        proceso_en_ejecucion = True
        codigos_procesados = 0
        estado_codigos = []
        await procesar_codigos(codigos)

# Ejecutar la función iniciar_procesamiento en el loop de eventos actual
async def main():
    await iniciar_procesamiento(codigos)

# Iniciar el procesamiento
await main()
